# This notebook requires a deployment of "gpt-4o-mini", edit as needed

## Web Search Ant As Tool

This example demonstrates one ant using another ant as a tool so that the second ant's work is outside the main thread.

The **WebSearchAgentAnt** demonstrates the use of a Web API secured using an API key with an auth header. 
This sample requires you to create a Web Search (bing) resource in Azure and enter its corresponding API key in [0-AI-settings](0-AI-settings.ipynb)

WebSearchAgentAnt uses **WebPageAgentAnt** via a tool call to evaluate search results using the search results link.
This demo uses a lot of tokens. 

**If your quota is low, you will get throttled**

## First steps
- First, download AntRunLib from Nuget
- At least once, set up the environment using **[0-AI-settings](0-AI-settings.ipynb)**

In [1]:
#r "nuget: AntRunnerLib, 0.9.6"

using AntRunnerLib;
using AntRunnerLib.Identity;
using static AntRunnerLib.ClientUtility;
using System.IO;

#!import config/Settings.cs 

var envVariables = Settings.GetEnvironmentVariables();
foreach (var kvp in envVariables)
{
    Environment.SetEnvironmentVariable(kvp.Key, kvp.Value);
}
Environment.SetEnvironmentVariable("AZURE_OPENAI_DEPLOYMENT", "gpt-4o-mini");

var config = AzureOpenAiConfigFactory.Get();
var client = GetOpenAiClient(config);


Installed Packages AntRunnerLib, 0.9.6

In [2]:
var assistantId = await AssistantUtility.Create("WebSearchAgentAnt", config);
Console.WriteLine(assistantId);
assistantId = await AssistantUtility.Create("WebPageAgentAnt", config);
Console.WriteLine(assistantId);

asst_MYZmH7FaPQgJBt6bNEIMOPQT
asst_LMfareDSr5U3WDbfRaVsxtOV


In [3]:
AntLoader.LoadAssembly(@".\AssistantDefinitions\WebPageAgentAnt\HtmlAgilityPack.dll");
AntLoader.LoadAssembly(@".\AssistantDefinitions\WebPageAgentAnt\HtmlAgilityPackExtensions.dll");

## Run the Assistant

`output.Dialog` shows the conversation.

In [4]:
var assistantRunOptions = new AssistantRunOptions() {
    AssistantName = "WebSearchAgentAnt",
    Instructions = "Use python to get todays date and then use it to find things to do in Atlanta today. Be careful to not include events in the past in your final answer.",
};
var output = await AntRunnerLib.AssistantRunner.RunThread(assistantRunOptions, config);
output.Dialog.DisplayAs("text/markdown");


User: Use python to get todays date and then use it to find things to do in Atlanta today. Be careful to not include events in the past in your final answer.

Assistant: I ran this code:
```
from datetime import datetime

# Get today's date
today_date = datetime.now().date()
today_date
```

I called the tool named crawl with these arguments:
```
{"q":"things to do in Atlanta January 13 2025","count":5,"textFormat":"Raw"}
```
and got this result:
```
{"queryContext":{"originalQuery":"things to do in Atlanta January 13 2025"},"webPages":{"value":[{"name":"Atlanta Events in January 2025 | Atlanta, GA","url":"https://atlanta-ga.events/january/","displayUrl":"https://atlanta-ga.events/january","snippet":"Atlanta Events January 2025. Check out the complete schedule of this month\u2019s events. ... Sports Events: 13 . Mercedes-Benz Stadium . 30313, Martin Luther King Jr Dr SW \u0026 Northside Dr NW, Atlanta, GA, US . Sports Events: 30 . Russ Chandler Stadium . 30318, 255 Ferst Drive Northwest, Atlanta, GA, US ..."},{"name":"Events \u0026 Things To Do in Atlanta in January 2025 - AllEvents.in","url":"https://allevents.in/atlanta/january","displayUrl":"https://allevents.in/atlanta/january","snippet":"Find events and things to do in January 2025 in Atlanta. Discover parties, concerts, meets, shows, sports, club, reunion, Performance happening in January 2025 in Atlanta. ... Mon, 13 Jan 328 interested. Free events in january in Atlanta. Best Rooftop Party in Atlanta! Suite Lounge with City View Clubs Atl VIP"},{"name":"Atlanta Events Calendar 2025/2026 | Atlanta, GA","url":"https://atlanta-ga.events/","displayUrl":"https://atlanta-ga.events","snippet":"This calendar is your ultimate guide to the best things to do in Atlanta, GA: ... Jan 13, 2025 3:30 AM . Chrisette Michele . Events: 12. City Winery - Atlanta . Jan 14, 2025 6:30 PM . Phoenix Suns . Events: 3. State Farm Arena - GA ...","thumbnailUrl":"https://www.bing.com/th?id=OIP.ytkv04WKv8RTyYo4_DsP4gHaHa\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Atlanta Events \u2013 January 2025 - Go South Atlanta","url":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-january.html","displayUrl":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-january.html","snippet":"\u2013 Atlanta Boat Show, January 9-12 2025 Annual boating trade show at the Georgia World Congress Center. More details \u2013 Scott Antique Market, January 9-12 2025 ... Events From January 13 To Martin Luther King Day Weekend, January 18-20 2025. \u2192 For concerts this week, see concerts in Atlanta in January \u2013 IllumiNights, November 14 2024 ...","thumbnailUrl":"https://www.bing.com/th?id=OIP.xm-3dn3hYJZHsVVU9WO5qwHaD4\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Things to do this weekend in metro Atlanta | Jan. 10-12, 2025","url":"https://www.fox5atlanta.com/news/things-do-weekend-metro-atlanta-jan-10-12-2025","displayUrl":"https://www.fox5atlanta.com/news/things-do-weekend-metro-atlanta-jan-10-12-2025","snippet":"Published January 9, 2025 2:43pm EST Things To Do. FOX 5 Atlanta ATLANTA - ... 6:30 p.m. Jan. 13 Where: ... Things To Do Atlanta Fulton County DeKalb County Gwinnett County Entertainment News.","thumbnailUrl":"https://www.bing.com/th?id=OIF.0x3UIPwUg5BH11ZgshIgMw\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"}]}}
```

I called the tool named ExtractContentFromUrl with these arguments:
```
{"assistantName": "WebPageAgentAnt", "instructions": "What events are happening in Atlanta on January 13, 2025? https://atlanta-ga.events/january/"}
```
and got this result:
```
"On January 13, 2025, the following event is happening in Atlanta:\n\n- **Titanic: An Immersive Voyage**\n  - **Time:** 3:30 AM\n  - **Location:** Exhibition Hub Art Center Atlanta, 5660 Buford Hwy NE, Atlanta, GA, US\n  - **Ticket Prices:** Prices from $55, average price approximately $65\n  - **Availability:** 32 tickets remaining\n  - **More Info:** [Titanic: An Immersive Voyage Tickets](https://atlanta-ga.events/tickets/titanic-an-immersive-voyage/)\n\n![Titanic: An Immersive Voyage](https://atlanta-ga.events/wp-content/uploads/2024/12/Titanic-An-Immersive-Voyage-Tickets.jpg)\n\nThis is the only listed event on that date."
```

I called the tool named ExtractContentFromUrl with these arguments:
```
{"assistantName": "WebPageAgentAnt", "instructions": "What events are happening in Atlanta on January 13, 2025? https://allevents.in/atlanta/january"}
```
and got this result:
```
"On January 13, 2025, the following event is happening in Atlanta:\n\n### Atlanta Career Fair\n- **Date:** January 13, 2025\n- **Location:** Hilton\n- **Interest Level:** 332 people are interested.\n\nFor more details, you can check the event page [here](https://allevents.in/atlanta/atlanta-career-fair/10000300752598117?ref=category-buckets).\n\nIf you\u0027re looking for more events or specific types of activities, feel free to ask!"
```

I called the tool named ExtractContentFromUrl with these arguments:
```
{"assistantName": "WebPageAgentAnt", "instructions": "What events are happening in Atlanta on January 13, 2025? https://atlanta-ga.events/"}
```
and got this result:
```
"On January 13, 2025, the following event is scheduled in Atlanta:\n\n- **Titanic: An Immersive Voyage**\n  - **Location:** Exhibition Hub Art Center Atlanta\n  - **Time:** 3:30 AM\n  - **Price:** Prices from $55 (average price ~ $65)\n  - **Tickets Remaining:** 32 tickets available\n  - [Get Tickets](https://atlanta-ga.events/event/titanic-an-immersive-voyage-atlanta-monday-january-13-2025-tba/?pid=8451)\n\n![Titanic: An Immersive Voyage](https://atlanta-ga.events/wp-content/uploads/2024/12/Titanic-An-Immersive-Voyage-Tickets.jpg)"
```

I called the tool named ExtractContentFromUrl with these arguments:
```
{"assistantName": "WebPageAgentAnt", "instructions": "What events are happening in Atlanta on January 13, 2025? https://gosouthatlanta.com/things-to-do/events/atlanta-in-january.html"}
```
and got this result:
```
"On January 13, 2025, the following events are happening in Atlanta:\n\n- **IllumiNights**: An annual after-hours walk-through holiday lights show at Zoo Atlanta, featuring a mile of illuminated animal lanterns. [More details](https://zooatlanta.org/event/illuminights/)\n\n- **Skate the Station**: An annual outdoor holiday skating event at Atlantic Station, running until January 20, 2025, with themed nights. [More details](https://atlanticstation.com/event/skate-the-station-2/)\n\n- **Love\u0027s Labour\u0027s Lost**: A play at the Shakespeare Tavern Playhouse, running from January 11 to January 26, 2025. [More details](https://www.shakespearetavern.com/on-stage/company/loves-labours-lost/)\n\nYou might also want to check out events happening just before and after this date for additional options."
```

I called the tool named ExtractContentFromUrl with these arguments:
```
{"assistantName": "WebPageAgentAnt", "instructions": "What events are happening in Atlanta on January 13, 2025? https://www.fox5atlanta.com/news/things-do-weekend-metro-atlanta-jan-10-12-2025"}
```
and got this result:
```
"On January 13, 2025, there is a trivia night event happening in Atlanta:\n\n- **Event**: Super Geek Trivia\n- **When**: 6:30 p.m. January 13\n- **Where**: Line Creek Brewing Bus Barn, 300 City Center Parkway, Fayetteville\n- **What**: Weekly trivia night at Line Creek Brewing Bus Barn. Enjoy fun questions and grab a glizzy from Mr. Perro\u0027s Hot Dogs.\n- **How Much**: Free admission\n\nYou can find more details [here](https://www.facebook.com/events/1983131175500957/1983131222167619/?active_tab=about)."
```

Here are some exciting events happening in Atlanta on January 13, 2025:

1. **Titanic: An Immersive Voyage**
   - **Time:** 3:30 AM
   - **Location:** Exhibition Hub Art Center Atlanta, 5660 Buford Hwy NE, Atlanta, GA, US
   - **Ticket Prices:** Starting from $55 (average price approximately $65)
   - **Availability:** 32 tickets remaining
   - **More Info:** [Titanic: An Immersive Voyage Tickets](https://atlanta-ga.events/tickets/titanic-an-immersive-voyage/)
   - ![Titanic: An Immersive Voyage](https://atlanta-ga.events/wp-content/uploads/2024/12/Titanic-An-Immersive-Voyage-Tickets.jpg)

2. **Atlanta Career Fair**
   - **Location:** Hilton
   - **Interest Level:** 332 people are interested.
   - **More Info:** [Atlanta Career Fair](https://allevents.in/atlanta/atlanta-career-fair/10000300752598117?ref=category-buckets)

3. **IllumiNights**
   - An annual after-hours walk-through holiday lights show at Zoo Atlanta, featuring a mile of illuminated animal lanterns.
   - [More details](https://zooatlanta.org/event/illuminights/)

4. **Skate the Station**
   - An outdoor holiday skating event at Atlantic Station, running until January 20, 2025, with themed nights.
   - [More details](https://atlanticstation.com/event/skate-the-station-2/)

5. **Love's Labour's Lost**
   - A play at the Shakespeare Tavern Playhouse, running from January 11 to January 26, 2025.
   - [More details](https://www.shakespearetavern.com/on-stage/company/loves-labours-lost/)

6. **Super Geek Trivia**
   - **When:** 6:30 PM
   - **Where:** Line Creek Brewing Bus Barn, 300 City Center Parkway, Fayetteville
   - **What:** Weekly trivia night at Line Creek Brewing Bus Barn. Enjoy fun questions and grab a glizzy from Mr. Perro's Hot Dogs.
   - **How Much:** Free admission
   - **More Info:** [Super Geek Trivia Event](https://www.facebook.com/events/1983131175500957/1983131222167619/?active_tab=about)

These events offer a variety of activities, from immersive experiences to career opportunities and entertainment. Enjoy your day in Atlanta!


In [5]:
output.LastMessage.DisplayAs("text/markdown");

Here are some exciting events happening in Atlanta on January 13, 2025:

1. **Titanic: An Immersive Voyage**
   - **Time:** 3:30 AM
   - **Location:** Exhibition Hub Art Center Atlanta, 5660 Buford Hwy NE, Atlanta, GA, US
   - **Ticket Prices:** Starting from $55 (average price approximately $65)
   - **Availability:** 32 tickets remaining
   - **More Info:** [Titanic: An Immersive Voyage Tickets](https://atlanta-ga.events/tickets/titanic-an-immersive-voyage/)
   - ![Titanic: An Immersive Voyage](https://atlanta-ga.events/wp-content/uploads/2024/12/Titanic-An-Immersive-Voyage-Tickets.jpg)

2. **Atlanta Career Fair**
   - **Location:** Hilton
   - **Interest Level:** 332 people are interested.
   - **More Info:** [Atlanta Career Fair](https://allevents.in/atlanta/atlanta-career-fair/10000300752598117?ref=category-buckets)

3. **IllumiNights**
   - An annual after-hours walk-through holiday lights show at Zoo Atlanta, featuring a mile of illuminated animal lanterns.
   - [More details](https://zooatlanta.org/event/illuminights/)

4. **Skate the Station**
   - An outdoor holiday skating event at Atlantic Station, running until January 20, 2025, with themed nights.
   - [More details](https://atlanticstation.com/event/skate-the-station-2/)

5. **Love's Labour's Lost**
   - A play at the Shakespeare Tavern Playhouse, running from January 11 to January 26, 2025.
   - [More details](https://www.shakespearetavern.com/on-stage/company/loves-labours-lost/)

6. **Super Geek Trivia**
   - **When:** 6:30 PM
   - **Where:** Line Creek Brewing Bus Barn, 300 City Center Parkway, Fayetteville
   - **What:** Weekly trivia night at Line Creek Brewing Bus Barn. Enjoy fun questions and grab a glizzy from Mr. Perro's Hot Dogs.
   - **How Much:** Free admission
   - **More Info:** [Super Geek Trivia Event](https://www.facebook.com/events/1983131175500957/1983131222167619/?active_tab=about)

These events offer a variety of activities, from immersive experiences to career opportunities and entertainment. Enjoy your day in Atlanta!

In [6]:
assistantRunOptions = new AssistantRunOptions() {
    AssistantName = "WebSearchAgentAnt",
    Instructions = "Use python to get todays date and then use it to find things to do in Atlanta for each of the next four weekends with pictures and correct links from ExtractContentFromUrl.\n\nWhen choosing links for crawl to evaluate with ExtractContentFromUrl choose a variety of hosts for urls from which you extract content. Limit your use of ExtractContentFromUrl to five calls or less. Do not make the error of including the url as a parameter, it must be inside 'instructions' as indicated by the tool definition. Be careful to not include events in the past in your final answer. You should perform four searches at once.\nDon't forget to include valid links!",
};
var output2 = await AntRunnerLib.AssistantRunner.RunThread(assistantRunOptions, config);
output2.Dialog.DisplayAs("text/markdown");


User: Use python to get todays date and then use it to find things to do in Atlanta for each of the next four weekends with pictures and correct links from ExtractContentFromUrl.

When choosing links for crawl to evaluate with ExtractContentFromUrl choose a variety of hosts for urls from which you extract content. Limit your use of ExtractContentFromUrl to five calls or less. Do not make the error of including the url as a parameter, it must be inside 'instructions' as indicated by the tool definition. Be careful to not include events in the past in your final answer. You should perform four searches at once.
Don't forget to include valid links!

Assistant: I ran this code:
```
from datetime import datetime, timedelta

# Get today's date
today = datetime.now()

# Calculate the next four weekends (Saturday)
next_four_weekends = [today + timedelta(days=(5 - today.weekday() + i * 7)) for i in range(4)]
next_four_weekends
```

I called the tool named crawl with these arguments:
```
{"q": "things to do in Atlanta January 18 2025", "count": 5, "textFormat": "Raw"}
```
and got this result:
```
{"queryContext":{"originalQuery":"things to do in Atlanta January 18 2025"},"webPages":{"value":[{"name":"Atlanta Events in January 2025 | Atlanta, GA","url":"https://atlanta-ga.events/january/","displayUrl":"https://atlanta-ga.events/january","snippet":"Atlanta Events January 2025. Check out the complete schedule of this month\u2019s events. ... 18 tickets remaining! Tickets; Jan. 11. 2025. 9:30 PM Sat. Howard Hewett. City Winery - Atlanta | Capacity: 300. 30308, 675 Ponce De Leon Ave, Atlanta, GA, US Prices from $91","thumbnailUrl":"https://www.bing.com/th?id=OIP.ytkv04WKv8RTyYo4_DsP4gHaHa\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Atlanta Events \u2013 January 2025 - Go South Atlanta","url":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-january.html","displayUrl":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-january.html","snippet":"Things to do in Atlanta, GA in January: calendar of upcoming festivals, Martin Luther King Day events, comedy, musicals, races, conventions, and other events. Go South! ... January 18-20 2025. \u2192 For concerts this week, see concerts in Atlanta in January \u2013 IllumiNights, November 14 2024 - January 19 2025 ...","thumbnailUrl":"https://www.bing.com/th?id=OIP.xm-3dn3hYJZHsVVU9WO5qwHaD4\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Events \u0026 Things To Do in Atlanta in January 2025 - AllEvents.in","url":"https://allevents.in/atlanta/january","displayUrl":"https://allevents.in/atlanta/january","snippet":"Find events and things to do in January 2025 in Atlanta. Discover parties, concerts, meets, shows, sports, club, reunion, Performance happening in January 2025 in Atlanta. ... Sat, 18 Jan Onwards Trending events in january in Atlanta. ELEVATE : IGNITE CONNECTIONS FOR 2025 The Ritz-Carlton, Atlanta Sat, 11 Jan ..."},{"name":"20 Absolutely Fantastic Things To Do In Atlanta This January","url":"https://secretatlanta.co/things-to-do-january/","displayUrl":"https://secretatlanta.co/things-to-do-january","snippet":"If you\u2019re looking for some fun things to do around Atlanta this January, don\u2019t worry, we\u2019ve got ya covered! ... Woodruff Park | 91 Peachtree Street, Atlanta, GA. When: Until January 12th, 2025. If you\u2019re searching for a dazzling display to check out this January, don\u2019t worry, we\u2019ve got you covered! ... 18. Set sail on an emotional ...","thumbnailUrl":"https://www.bing.com/th?id=OIP.5lOsdA8tsTOgx5qTd465YAAAAA\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Cool Things To Do in January in Atlanta","url":"https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-january-in-atlanta/","displayUrl":"https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-january-in-atlanta","snippet":"Published December 17, 2024 Last Updated January 7, 2025. The holidays are over, but that doesn\u2019t mean there isn\u2019t plenty to celebrate. Whether you\u2019re in the mood for a show, a sporting event, something to do with the family or just looking for other fun festivities around Atlanta, there\u2019s a variety of things to do this January.","thumbnailUrl":"https://www.bing.com/th?id=OIP.aU9EcatRnz4MDkIKyYbkQgHaD4\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"}]}}
```

I called the tool named crawl with these arguments:
```
{"q": "things to do in Atlanta January 25 2025", "count": 5, "textFormat": "Raw"}
```
and got this result:
```
{"queryContext":{"originalQuery":"things to do in Atlanta January 25 2025"},"webPages":{"value":[{"name":"Atlanta Events in January 2025 | Atlanta, GA","url":"https://atlanta-ga.events/january/","displayUrl":"https://atlanta-ga.events/january","snippet":"Atlanta Events January 2025. ... Jan 16, 2025 7:00 PM . Show More. Concerts Venues. List of all concert venues in the city and Atlanta area. ... Shows \u0026 Events: 25 . Show More. Sports Venues. List of all sports arenas and stadiums in the city and Atlanta area. Atlanta Motor Speedway . 30228, 1500 Tara Place, Hampton, GA, US ...","thumbnailUrl":"https://www.bing.com/th?id=OIP.ytkv04WKv8RTyYo4_DsP4gHaHa\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Atlanta Events \u2013 January 2025 - Go South Atlanta","url":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-january.html","displayUrl":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-january.html","snippet":"Things to do in Atlanta, GA in January: calendar of upcoming festivals, Martin Luther King Day events, comedy, musicals, races, conventions, and other events. ... January 25 2025 Special event at the Atlanta Botanical Garden, with speakers and plant sale. More details \u2013 Gals on the Go, January 25 2025","thumbnailUrl":"https://www.bing.com/th?id=OIP.xm-3dn3hYJZHsVVU9WO5qwHaD4\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Events \u0026 Things To Do in Atlanta in January 2025 - AllEvents.in","url":"https://allevents.in/atlanta/january","displayUrl":"https://allevents.in/atlanta/january","snippet":"Find events and things to do in January 2025 in Atlanta. Discover parties, concerts, meets, shows, sports, club, reunion, Performance happening in January 2025 in Atlanta. Open Menu. Atlanta Change City; ... 24-25 Jan USD 10 Atlanta Exotic Paintings Nude Sip \u0026 Paint 1730 Timothy Dr SW 10 Jan Onwards USD 25 ..."},{"name":"25 Reasons to Visit Atlanta in 2025","url":"https://discoveratlanta.com/stories/things-to-do/25-reasons-to-visit-atlanta-in-2025/","displayUrl":"https://discoveratlanta.com/stories/things-to-do/25-reasons-to-visit-atlanta-in-2025","snippet":"The Atlanta Chinese Dance Company performs on stage on Jan. 25,2025, at Aurora Children\u2019s Playhouse. Here are more ways to celebrate Chinese New Year in Atlanta. Spring Festivals. Atlanta Film Festival 2025 takes place April 24 through May 4, 2025, at the Plaza Theatre \u0026 Tara Theatre."},{"name":"20 free things to do in metro Atlanta in January","url":"https://www.accessatlanta.com/atlanta-things-to-do/20-free-things-to-do-in-metro-atlanta-in-january/Q3EOZY6OHBE2HLT4QN5ZIBLBQU/","displayUrl":"https://www.accessatlanta.com/atlanta-things-to-do/20-free-things-to-do-in-metro...","snippet":"Here are 20 of the best things to do in Atlanta in January that are totally free. ... there\u2019s no shortage of opportunities to kick-start 2025 with purpose and fun. ... Saturday, Jan. 25; Time: 9 ..."}]}}
```

I called the tool named crawl with these arguments:
```
{"q": "things to do in Atlanta February 1 2025", "count": 5, "textFormat": "Raw"}
```
and got this result:
```
{"queryContext":{"originalQuery":"things to do in Atlanta February 1 2025"},"webPages":{"value":[{"name":"Atlanta Events February 2025 | Atlanta, GA","url":"https://atlanta-ga.events/february/","displayUrl":"https://atlanta-ga.events/february","snippet":"Atlanta Events February 2025. Check out the complete schedule of this month\u2019s events. ... Feb 1, 2025 8:00 PM . The Day Players . Events: 2. Helium Comedy Club - Atlanta . Feb 2, 2025 1:00 PM . Both Sides Now: The Music of Joni Mitchell and Leonard Cohen ...","thumbnailUrl":"https://www.bing.com/th?id=OIP.ytkv04WKv8RTyYo4_DsP4gHaHa\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Events in Atlanta - February 2025 - Go South Atlanta","url":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-february.html","displayUrl":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-february.html","snippet":"\u2013 The Sweet Spot Burlesque: Masquerade Edition, February 1 2025 Burlesque performance at Center Stage. More details \u2013 Round Table Game Show, February 1 2025 Comedy show at Center Stage. ... More Things To Do In Atlanta In February \u2013 State parks and nature sights near Atlanta \u2013 Black history sites in Atlanta \u2013 Museums and historic houses","thumbnailUrl":"https://www.bing.com/th?id=OIP.xm-3dn3hYJZHsVVU9WO5qwHaD4\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Events \u0026 Things To Do In Atlanta on 1st February 2025 - AllEvents.in","url":"https://allevents.in/atlanta/2025-02-01","displayUrl":"https://allevents.in/atlanta/2025-02-01","snippet":"Discover the best events and activities in Atlanta on 1st February 2025. From live concerts to art exhibitions, find the perfect thing to do for an unforgettable day out. ... Events \u0026 Things To Do In January 2025 In Atlanta Live Music Events Near Atlanta Events In Atlanta Today Baltimore Events Houston Events Las Vegas Events ..."},{"name":"Explore Atlanta Events for Today, This Week or Weekend","url":"https://discoveratlanta.com/events/main/","displayUrl":"https://discoveratlanta.com/events","snippet":"Explore the Atlanta events calendar for things to do today, this week or this weekend. Find upcoming festivals, seasonal events and more! ... Maternal Health Awareness Day 2025 1/23/25. Movies at The Strand: Anastasia (1997) 1/23/25. Winter Game Dinner ... February 12. Wednesday. Alvin Ailey American Dance Theater Live at the Fox 2/12/25 \u2013 2 ...","thumbnailUrl":"https://www.bing.com/th?id=OIP.FkxSV6JbrWatsFMM3_8dIAHaEh\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"21 Fabulous Things To Do In Atlanta This February - Secret Atlanta","url":"https://secretatlanta.co/things-to-do-february/","displayUrl":"https://secretatlanta.co/things-to-do-february","snippet":"We made a breakdown of 21 of the best things to do this February in Atlanta! 1. Celebrate and honor Black History Month ... January 9, 2025. Stone Mountain\u2019s Iconic Lunar New Year Festival Is Back For The Year Of The Snake. January 9, 2025. SHARE THE ARTICLE. Secret Guides Things To Do.","thumbnailUrl":"https://www.bing.com/th?id=OIP.E9ip87TaH0u1SNr7-EpqJwHaE8\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"}]}}
```

I called the tool named crawl with these arguments:
```
{"q": "things to do in Atlanta February 8 2025", "count": 5, "textFormat": "Raw"}
```
and got this result:
```
{"queryContext":{"originalQuery":"things to do in Atlanta February 8 2025"},"webPages":{"value":[{"name":"Atlanta Events February 2025 | Atlanta, GA","url":"https://atlanta-ga.events/february/","displayUrl":"https://atlanta-ga.events/february","snippet":"Feb 4, 2025 8:00 PM . Lutalo . Events: 2. The Earl ... Atlanta Events 2025/2026 . Contact Us. Stay up to date! Join our newsletter to stay current on all Live Entertainment in Atlanta, plus the occasional special offer, VIP packages, and discount tickets! Unique events and incredible deals are straight to your inbox.","thumbnailUrl":"https://www.bing.com/th?id=OIP.ytkv04WKv8RTyYo4_DsP4gHaHa\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Events in Atlanta - February 2025 - Go South Atlanta","url":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-february.html","displayUrl":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-february.html","snippet":"Things to do in Atlanta, GA in February, including upcoming festivals, comedy, musicals, Presidents Day weekend events, conventions, shows, and other events. Go South! ... January 30 - February 8 2025 Musical at Marietta\u2019s New Theatre in the Square. More details \u2013 Shen Yun, January 31 - February 2 2025 Dance performance at the Cobb Energy ...","thumbnailUrl":"https://www.bing.com/th?id=OIP.xm-3dn3hYJZHsVVU9WO5qwHaD4\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Discover February Events \u0026 Activities in Atlanta, GA - Eventbrite","url":"https://www.eventbrite.com/d/ga--atlanta/february/","displayUrl":"https://www.eventbrite.com/d/ga--atlanta/february","snippet":"Share this event: BUILDING DEPTH 5x5 - FEB 7-8, 2025 (ATLANTA, GA) General Admission Save this event: BUILDING DEPTH 5x5 - FEB 7-8, 2025 (ATLANTA, GA) General Admission. R\u0026B ONLY LIVE - Atlanta, GA (10 Year Anniversary) Sat, Feb 22, 8:00 PM. The Eastern."},{"name":"21 Fabulous Things To Do In Atlanta This February - Secret Atlanta","url":"https://secretatlanta.co/things-to-do-february/","displayUrl":"https://secretatlanta.co/things-to-do-february","snippet":"We made a breakdown of 21 of the best things to do this February in Atlanta! 1. Celebrate and honor Black History Month ... January 9, 2025. Stone Mountain\u2019s Iconic Lunar New Year Festival Is Back For The Year Of The Snake. January 9, 2025. SHARE THE ARTICLE. Secret Guides Things To Do.","thumbnailUrl":"https://www.bing.com/th?id=OIP.E9ip87TaH0u1SNr7-EpqJwHaE8\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Events \u0026 Things To Do in Atlanta in February 2025 - AllEvents.in","url":"https://allevents.in/atlanta/february","displayUrl":"https://allevents.in/atlanta/february","snippet":"The Valentine\u2019s week surely comes in with a bunch of things to do in Atlanta in February 2025. Not just Valentine\u2019s there are all the more festivals, concerts and events in February in Atlanta waiting to be discovered. Look for this space to find out a curated list for February 2025 events in Atlanta"}]}}
```

I called the tool named ExtractContentFromUrl with these arguments:
```
{"assistantName": "WebPageAgentAnt", "instructions": "What are the events happening in Atlanta on January 18, 2025? https://atlanta-ga.events/january/"}
```
and got this result:
```
"On January 18, 2025, the following events are scheduled to take place in Atlanta:\n\n1. **Titanic: An Immersive Voyage**\n   - **Time**: 3:30 AM\n   - **Location**: Exhibition Hub Art Center Atlanta\n   - **Prices**: From $64, Average price ~ $77\n   - ![Titanic: An Immersive Voyage](https://atlanta-ga.events/wp-content/uploads/2024/12/Titanic-An-Immersive-Voyage-Tickets.jpg)\n   - [More Info \u0026 Tickets](https://atlanta-ga.events/tickets/titanic-an-immersive-voyage/)\n\n2. **Kennesaw State Owls vs. Louisiana Tech Bulldogs**\n   - **Time**: 5:00 PM\n   - **Location**: KSU Convocation Center\n   - **Prices**: From $26, Average price ~ $32\n   - ![Kennesaw State Owls vs. Louisiana Tech Bulldogs](https://atlanta-ga.events/wp-content/uploads/2024/02/kennesaw-state-owls-basketball.jpg)\n   - [More Info \u0026 Tickets](https://atlanta-ga.events/event/kennesaw-state-owls-vs-louisiana-tech-bulldogs-kennesaw-saturday-january-18-2025-500-pm/?pid=4812)\n\n3. **X-Cops**\n   - **Time**: 6:00 PM\n   - **Location**: Purgatory Stage at Masquerade\n   - **Prices**: From $23, Average price ~ $32\n   - ![X-Cops](https://atlanta-ga.events/wp-content/uploads/2024/11/Atlanta-Events-Schedule.png)\n   - [More Info \u0026 Tickets](https://atlanta-ga.events/event/x-cops-atlanta-saturday-january-18-2025-600-pm/?pid=7908)\n\n4. **Tacarra Williams**\n   - **Time**: 7:00 PM\n   - **Location**: ATL Comedy Theater\n   - **Prices**: From $67, Average price ~ $77\n   - ![Tacarra Williams](https://atlanta-ga.events/wp-content/uploads/2024/10/tacarra-williams.jpg)\n   - [More Info \u0026 Tickets](https://atlanta-ga.events/event/tacarra-williams-norcross-saturday-january-18-2025-700-pm/?pid=8003)\n\n5. **Greenville Swamp Rabbits**\n   - **Time**: 7:00 PM\n   - **Location**: Gas South Arena\n   - **Prices**: From $27, Average price ~ $32\n   - ![Greenville Swamp Rabbits](https://atlanta-ga.events/wp-content/uploads/2024/02/greenville-swamp-rabbits.jpg)\n   - [More Info \u0026 Tickets](https://atlanta-ga.events/event/greenville-swamp-rabbits-atlanta-saturday-january-18-2025-700-pm/?pid=8004)\n\n6. **AT\u0026T Playoff Playlist Live**\n   - **Time**: 7:00 PM\n   - **Location**: State Farm Arena\n   - **Prices**: From $15, Average price ~ $25\n   - ![AT\u0026T Playoff Playlist Live](https://atlanta-ga.events/wp-content/uploads/2024/11/Atlanta-Events-Schedule.png)\n   - [More Info \u0026 Tickets](https://atlanta-ga.events/event/att-playoff-playlist-live-atlanta-saturday-january-18-2025-700-pm/?pid=8005)\n\n7. **Electric Avenue**\n   - **Time**: 8:00 PM\n   - **Location**: Variety Playhouse\n   - **Prices**: From $28, Average price ~ $32\n   - ![Electric Avenue](https://atlanta-ga.events/wp-content/uploads/2023/07/Electric-Avenue.jpg)\n   - [More Info \u0026 Tickets](https://atlanta-ga.events/event/electric-avenue-atlanta-saturday-january-18-2025-800-pm/?pid=8006)\n\nThese events cover a range of entertainment options from exhibitions, sports, comedy shows, to concerts."
```

I called the tool named ExtractContentFromUrl with these arguments:
```
{"assistantName": "WebPageAgentAnt", "instructions": "What are the events happening in Atlanta on January 25, 2025? https://gosouthatlanta.com/things-to-do/events/atlanta-in-january.html"}
```
and got this result:
```
"On January 25, 2025, several events are scheduled in Atlanta:\n\n1. **Run With the Dogs**: An annual 5K and pet races in Decatur to benefit the Decatur Bulldogs Boosters. [More details](http://www.rwtd5k.com)\n\n2. **Spring Gardening Symposium \u0026 Plant Sale**: A special event at the Atlanta Botanical Garden featuring speakers and a plant sale. [More details](https://atlantabg.org/calendar/spring-symposium-2025/)\n\n3. **Gals on the Go**: A live podcast show at Center Stage. [More details](https://www.centerstage-atlanta.com/events/gals-on-the-go/)\n\n4. **Maria Bamford**: A comedy show at the Buckhead Theatre. [More details](https://concerts.livenation.com/maria-bamford-atlanta-georgia-01-25-2025/event/0E00611B94352720)\n\n5. **Magician Rich Crawford**: An illusion show at Red Clay Music Foundry, Duluth. [More details](https://eddieowenpresents.freshtix.com/events/richcrawford)\n\n6. **Lunar New Year Festival**: An Asian-inspired festival at Stone Mountain Park, featuring a drone and light show, lighted parade, cultural craft activities, live performances, and other cultural activities. [More details](https://stonemountainpark.com/activity/events/lunar-new-year/)\n\nThese events offer a variety of activities, from outdoor runs to comedy shows and cultural festivals."
```

I called the tool named ExtractContentFromUrl with these arguments:
```
{"assistantName": "WebPageAgentAnt", "instructions": "What are the events happening in Atlanta on February 1, 2025? https://gosouthatlanta.com/things-to-do/events/atlanta-in-february.html"}
```
and got this result:
```
"On February 1, 2025, several events are happening in Atlanta:\n\n1. **Andrew Stanley** - Comedy show at the Buckhead Theatre. [More details](https://concerts.livenation.com/andrew-stanley-800p-comedy-special-taping-atlanta-georgia-02-01-2025/event/0E00611BF7ED694A)\n\n2. **The Sweet Spot Burlesque: Masquerade Edition** - Burlesque performance at Center Stage. [More details](https://www.centerstage-atlanta.com/events/the-sweet-spot-burlesque-masquerade-edition/)\n\n3. **Round Table Game Show** - Comedy show at Center Stage. [More details](https://www.centerstage-atlanta.com/events/round-table-game-show/)\n\n4. **Hot Wheels Monster Trucks Live** - Monster truck show at State Farm Arena (February 1-2, 2025). [More details](https://www.statefarmarena.com/events/detail/hot-wheels-monster-trucks-live-glow-n-fire)\n\n5. **Romeo and Juliet** - Play at the Shakespeare Tavern Playhouse (February 1 - March 2, 2025). [More details](https://www.shakespearetavern.com/on-stage/company/romeo-and-juliet/)\n\nThese events cater to a variety of interests, including comedy, performance arts, and family-friendly entertainment."
```

I called the tool named ExtractContentFromUrl with these arguments:
```
{"assistantName": "WebPageAgentAnt", "instructions": "What are the events happening in Atlanta on February 8, 2025? https://gosouthatlanta.com/things-to-do/events/atlanta-in-february.html"}
```
and got this result:
```
"On February 8, 2025, the following events are scheduled to take place in Atlanta:\n\n1. **Jeff Dunham: Artificial Intelligence**\n   - A comedy and ventriloquism show at the Fox Theatre.\n   - [More details](https://www.foxtheatre.org/events/detail/jeff-dunham)\n\n2. **MLW SuperFight 6**\n   - A wrestling event at Center Stage.\n   - [More details](https://www.centerstage-atlanta.com/events/mlw-superfight-6/)\n\n3. **The Rite of Spring: Atlanta Ballet**\n   - A ballet performance at the Cobb Energy Performing Arts Centre.\n   - [More details](https://www.cobbenergycentre.com/events/detail/the-rite-of-spring)\n\nThese events offer a mix of comedy, wrestling, and ballet for entertainment on that date."
```

Here are some exciting events happening in Atlanta over the next four weekends:

### January 18, 2025
1. **Titanic: An Immersive Voyage**
   - **Time**: 3:30 AM
   - **Location**: Exhibition Hub Art Center Atlanta
   - **Price**: From $64
   - ![Titanic: An Immersive Voyage](https://atlanta-ga.events/wp-content/uploads/2024/12/Titanic-An-Immersive-Voyage-Tickets.jpg)
   - [More Info & Tickets](https://atlanta-ga.events/tickets/titanic-an-immersive-voyage/)

2. **Kennesaw State Owls vs. Louisiana Tech Bulldogs**
   - **Time**: 5:00 PM
   - **Location**: KSU Convocation Center
   - **Price**: From $26
   - ![Kennesaw State Owls vs. Louisiana Tech Bulldogs](https://atlanta-ga.events/wp-content/uploads/2024/02/kennesaw-state-owls-basketball.jpg)
   - [More Info & Tickets](https://atlanta-ga.events/event/kennesaw-state-owls-vs-louisiana-tech-bulldogs-kennesaw-saturday-january-18-2025-500-pm/?pid=4812)

3. **X-Cops**
   - **Time**: 6:00 PM
   - **Location**: Purgatory Stage at Masquerade
   - **Price**: From $23
   - ![X-Cops](https://atlanta-ga.events/wp-content/uploads/2024/11/Atlanta-Events-Schedule.png)
   - [More Info & Tickets](https://atlanta-ga.events/event/x-cops-atlanta-saturday-january-18-2025-600-pm/?pid=7908)

4. **Tacarra Williams**
   - **Time**: 7:00 PM
   - **Location**: ATL Comedy Theater
   - **Price**: From $67
   - ![Tacarra Williams](https://atlanta-ga.events/wp-content/uploads/2024/10/tacarra-williams.jpg)
   - [More Info & Tickets](https://atlanta-ga.events/event/tacarra-williams-norcross-saturday-january-18-2025-700-pm/?pid=8003)

5. **Greenville Swamp Rabbits**
   - **Time**: 7:00 PM
   - **Location**: Gas South Arena
   - **Price**: From $27
   - ![Greenville Swamp Rabbits](https://atlanta-ga.events/wp-content/uploads/2024/02/greenville-swamp-rabbits.jpg)
   - [More Info & Tickets](https://atlanta-ga.events/event/greenville-swamp-rabbits-atlanta-saturday-january-18-2025-700-pm/?pid=8004)

### January 25, 2025
1. **Run With the Dogs**
   - An annual 5K and pet races in Decatur. [More details](http://www.rwtd5k.com)

2. **Spring Gardening Symposium & Plant Sale**
   - At the Atlanta Botanical Garden. [More details](https://atlantabg.org/calendar/spring-symposium-2025/)

3. **Gals on the Go**
   - A live podcast show at Center Stage. [More details](https://www.centerstage-atlanta.com/events/gals-on-the-go/)

4. **Maria Bamford**
   - Comedy show at the Buckhead Theatre. [More details](https://concerts.livenation.com/maria-bamford-atlanta-georgia-01-25-2025/event/0E00611B94352720)

5. **Lunar New Year Festival**
   - An Asian-inspired festival at Stone Mountain Park. [More details](https://stonemountainpark.com/activity/events/lunar-new-year/)

### February 1, 2025
1. **Andrew Stanley**
   - Comedy show at the Buckhead Theatre. [More details](https://concerts.livenation.com/andrew-stanley-800p-comedy-special-taping-atlanta-georgia-02-01-2025/event/0E00611BF7ED694A)

2. **The Sweet Spot Burlesque: Masquerade Edition**
   - Burlesque performance at Center Stage. [More details](https://www.centerstage-atlanta.com/events/the-sweet-spot-burlesque-masquerade-edition/)

3. **Hot Wheels Monster Trucks Live**
   - Monster truck show at State Farm Arena. [More details](https://www.statefarmarena.com/events/detail/hot-wheels-monster-trucks-live-glow-n-fire)

4. **Romeo and Juliet**
   - Play at the Shakespeare Tavern Playhouse. [More details](https://www.shakespearetavern.com/on-stage/company/romeo-and-juliet/)

### February 8, 2025
1. **Jeff Dunham: Artificial Intelligence**
   - Comedy and ventriloquism show at the Fox Theatre. [More details](https://www.foxtheatre.org/events/detail/jeff-dunham)

2. **MLW SuperFight 6**
   - Wrestling event at Center Stage. [More details](https://www.centerstage-atlanta.com/events/mlw-superfight-6/)

3. **The Rite of Spring: Atlanta Ballet**
   - Ballet performance at the Cobb Energy Performing Arts Centre. [More details](https://www.cobbenergycentre.com/events/detail/the-rite-of-spring)

These events offer a variety of entertainment options, from comedy shows and sports events to cultural festivals and immersive experiences. Enjoy your weekends in Atlanta!


## Clean Up

In [7]:
var assistant = (await client.AssistantList())?.Data?.FirstOrDefault(o => o.Name == "WebSearchAgentAnt");
if(assistant != null) {
    await client.AssistantDelete(assistant.Id);
    Console.WriteLine("Deleted assistant");
}
else
{
    Console.WriteLine("Didn't find WebSearchAgentAnt");
}

assistant = (await client.AssistantList())?.Data?.FirstOrDefault(o => o.Name == "WebPageAgentAnt");
if(assistant != null) {
    await client.AssistantDelete(assistant.Id);
    Console.WriteLine("Deleted assistant");
}
else
{
    Console.WriteLine("Didn't find WebPageAgentAnt");
}

Deleted assistant
Deleted assistant
